In [1]:
import time            
import re            
import os    
import sys  
import codecs  
import shutil  
import numpy as np  
import matplotlib  
import scipy  
import pandas as pd
import matplotlib.pyplot as plt  
from sklearn import feature_extraction    
from sklearn.feature_extraction.text import TfidfTransformer    
from sklearn.feature_extraction.text import CountVectorizer  
from sklearn.feature_extraction.text import HashingVectorizer

In [62]:
# #結巴 匯入自己的詞典
# jieba.set_dictionary('./dict/jieba_dict.txt.big')

#停止詞字典stopwordset
stopwordset = set()
with open('./dic/stopwords.txt', 'r', encoding='utf-8') as sw:
    for line in sw:
        stopwordset.add(line.strip('\n'))

In [70]:
#一个商品
review = open('../CN_20171130/pre/review.txt',encoding='utf-8').read().split('\n')
review[0]

'头可动，模型也很逼真|商品到达得很快，很不错~||||||||'

In [71]:
#一条评论
Reviews=[]
for rev in review:
    a = rev.split('|')
    if a !='':
        Reviews += a 
Reviews[:100]

['头可动，模型也很逼真',
 '商品到达得很快，很不错~',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '非常喜欢哦，卡斯利亚斯',
 'Kaecilius头又大又重……很难站起来啊……正考虑要不要搞点双面胶粘到桌子上！还有这个是摇头款啦 头和身体是用弹簧连接的 所以 Funko脑子是坏掉了么……好变态的设计（不',
 '好东西！！！喜欢拔叔的必入！！！！而且非中国造！！！',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '比较小，容组装，感觉还好吗',
 "This is a present for my 3 years old son.  He got interested in it. But for myself, I love it very much. The set contains all details about a camping and each piece is designed exquisitely. I  can play it with my son in many ways. We can build up all the pieces and my son loves to put the sticker on the little one. Also, we can play with it as we are take camping and I can tell story with all the set.I love it. It's the best set that I have even as a adult. You never get boring with the little furniture and  kitchen appliances.",
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '颜色和质量非常好，宝贝很喜欢',
 '这个很喜欢，送给女孩子的。很好。',
 '不值这个价  特别迷你啊',
 '质量不错，儿子爱玩，很好',
 '',
 '',
 '',
 '',
 '',
 '',
 '特别美！！头发做得超级精致！',
 '',
 '',
 '',
 '',
 '',
 '',
 '',

In [72]:
# 断词
import jieba  
import jieba.analyse  
corpus=[]
for review in Reviews:
#     if review !='':
    words=jieba.cut(review, cut_all=False)
    pincut=" ".join([word for word in words if word not in stopwordset and '\u4e00' <= word and word <= '\u9fff'])
    corpus.append(pincut)
# corpus =corpus[:-1]   
len(corpus)

6290

In [48]:
#title
asins = open('../CN_20171130/pre/ASIN.csv').read().split('\n')
ASINs=[]
for item in asins:
    a = item.split('\t')
    ASINs += a
# ASINs =ASINs[:-1]   
len(ASINs)

6290

In [49]:
#genres
categorys = open('../CN_20171130/pre/categories.csv').read().split('\n')

Category=[]
for cat in categorys:
    a = cat.split('\t')
    Category += a
# Category = Category[:-1]
len(Category)

6290

In [26]:
#ranks
ranks = open('../CN_20171130/pre/ranks.csv').read().split('\n')
# ranks[0] = ranks[0].replace('\ufeff','')
Rank =[]
for rank in ranks:
    a = rank.split('\t')
    Rank += a
    
Ranks =[float(i) for i in Rank]  
Ranks[0]
len(Ranks)

6290

## Tf-idf and document similarity

In [73]:
#将评论中的词语转换为词频矩阵 矩阵元素a[i][j] 表示j词在第i类评论下的词频  
vectorizer = CountVectorizer()

In [74]:
#该类会统计每个词语的tf-idf权值  
transformer = TfidfTransformer() 

In [75]:
tf = vectorizer.fit_transform(corpus)

In [76]:
#第一个fit_transform是计算tf-idf 第二个fit_transform是将文本转为词频矩阵  
tfidf = transformer.fit_transform(tf)

In [77]:
#获取词袋模型中的前10个词语    
word = vectorizer.get_feature_names()
word[:10]

['一丁点', '一上', '一下子', '一下子把', '一不小心', '一不留神', '一个个', '一个劲', '一个半', '一个多']

In [78]:
# 词袋模型中共有几个词语
len(word)

5360

In [79]:
#将tf-idf矩阵抽取出来，元素w[i][j]表示j词在i类文本中的tf-idf权重  
weight = tfidf.toarray() 
weight

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [80]:
#打印特征向量文本内容  
print ('Features length: ' + str(len(word))) 
resName = "BHTfidf_Result.txt"  
result = codecs.open(resName, 'w', 'utf-8')  
for j in range(len(word)):  
    result.write(word[j] + ' ')  
result.write('\r\n\r\n') 

Features length: 5360


In [81]:
from sklearn.metrics.pairwise import cosine_similarity
dist = 1 - cosine_similarity(tfidf)

In [82]:
print(dist)

[[ -2.22044605e-16   1.00000000e+00   1.00000000e+00 ...,   1.00000000e+00
    1.00000000e+00   1.00000000e+00]
 [  1.00000000e+00   0.00000000e+00   1.00000000e+00 ...,   1.00000000e+00
    1.00000000e+00   9.04035014e-01]
 [  1.00000000e+00   1.00000000e+00   1.00000000e+00 ...,   1.00000000e+00
    1.00000000e+00   1.00000000e+00]
 ..., 
 [  1.00000000e+00   1.00000000e+00   1.00000000e+00 ...,  -2.22044605e-16
    9.77831781e-01   8.81090336e-01]
 [  1.00000000e+00   1.00000000e+00   1.00000000e+00 ...,   9.77831781e-01
   -6.66133815e-16   1.00000000e+00]
 [  1.00000000e+00   9.04035014e-01   1.00000000e+00 ...,   8.81090336e-01
    1.00000000e+00   0.00000000e+00]]


# K-means clustering

In [83]:
from sklearn.cluster import KMeans  
num_clusters = 10
km = KMeans(n_clusters=10)   
%time s = km.fit(weight)  
clusters = km.labels_.tolist()
print(s)

CPU times: user 26.9 s, sys: 3.55 s, total: 30.4 s
Wall time: 25.2 s
KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=10, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)


In [22]:
# from scipy.spatial.distance import cdist
# X = weight #在weight矩阵中元素w[i][j]表示j词在i类文本中的tf-idf权重  
# K = range(1, 100)
# meandistortions = []
# for k in K:
#     kmeans = KMeans(n_clusters=k)
#     kmeans.fit(X)
#     # 求kmeans的成本函数值
#     meandistortions.append(sum(np.min(cdist(X, kmeans.cluster_centers_, 'euclidean'), axis=1)) / X.shape[0])

# plt.figure()
# plt.grid(True)
# plt1 = plt.subplot(2,1,1)
# # 画样本点
# plt1.plot(X[:,0],X[:,1],'k.');
# plt2 = plt.subplot(2,1,2)
# # 画成本函数值曲线
# plt2.plot(K, meandistortions, 'bx-')
# plt.show()

KeyboardInterrupt: 

In [84]:
from sklearn.externals import joblib
# 注释语句用来存储你的模型
# 因为我已经从 pickle 载入过模型了
 
joblib.dump(km,  'doc_cluster.pkl')
 
km = joblib.load('doc_cluster.pkl')
clusters = km.labels_.tolist()

In [86]:
toys = { 'title': ASINs, 'rank': Ranks, 'review': Reviews, 'cluster': clusters, 'category': Category }
 
frame = pd.DataFrame(toys , index = [clusters] , columns = ['rank', 'title', 'cluster', 'category'])

In [87]:
frame['cluster'].value_counts() #number of review per cluster (clusters from 0 to 10)

0    4171
4     484
5     463
3     350
2     305
8     162
7     142
6     109
9      65
1      39
Name: cluster, dtype: int64

In [88]:
km.inertia_

3764.3703662924549

In [89]:
grouped = frame['rank'].groupby(frame['cluster']) # 为了凝聚（aggregation），由聚类分类。
 
grouped.mean() # 每个聚类的平均排名（0 到 5）

cluster
0    1.899065
1    4.974359
2    4.757377
3    4.711429
4    4.708678
5    4.840173
6    4.513761
7    4.619718
8    4.851852
9    4.430769
Name: rank, dtype: float64

In [90]:
from __future__ import print_function
 
print("Top terms per cluster:")
print()
# 按离质心的距离排列聚类中心，由近到远
order_centroids = km.cluster_centers_.argsort()[:, ::-1] 
 
for i in range(num_clusters):
    print("Cluster %d titles:" % i, end='')
    for title in frame.ix[i]['title'].values.tolist():
        print(' %s,' % title, end='')
    print() # 空行
    print() # 空行

Top terms per cluster:

Cluster 0 titles: "B06WWD7SMQ_1, B06WWD7SMQ_3, B06WWD7SMQ_4, B06WWD7SMQ_5, B06WWD7SMQ_6, B06WWD7SMQ_7, B06WWD7SMQ_8, B06WWD7SMQ_9, B06WWD7SMQ_10", "B01JL8KELG_1, B01JL8KELG_2, B01JL8KELG_3, B01JL8KELG_4, B01JL8KELG_5, B01JL8KELG_6, B01JL8KELG_7, B01JL8KELG_8, B01JL8KELG_9, B01JL8KELG_10", "B00A30ZA34_1, B00A30ZA34_2, B00A30ZA34_3, B00A30ZA34_4, B00A30ZA34_5, B00A30ZA34_6, B00A30ZA34_7, B00A30ZA34_8, B00A30ZA34_9, B00A30ZA34_10", B00VR2EB3W_2, B00VR2EB3W_5, B00VR2EB3W_6, B00VR2EB3W_7, B00VR2EB3W_8, B00VR2EB3W_9, B00VR2EB3W_10", B019JIADG4_2, B019JIADG4_3, B019JIADG4_4, B019JIADG4_5, B019JIADG4_6, B019JIADG4_7, B019JIADG4_8, B019JIADG4_9, B019JIADG4_10", B0063DEIQK_2, B0063DEIQK_4, B0063DEIQK_5, B0063DEIQK_6, B0063DEIQK_7, B0063DEIQK_8, B0063DEIQK_9, B0063DEIQK_10", B01MZALP0W_2, B01MZALP0W_5, B01MZALP0W_7, B01MZALP0W_10", B06XHJ1JRT_2, B06XHJ1JRT_3, B06XHJ1JRT_4, B06XHJ1JRT_5, B06XHJ1JRT_6, B06XHJ1JRT_7, B06XHJ1JRT_8, B06XHJ1JRT_9, B06XHJ1JRT_10", B00U7EX51K_2, B

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:10: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  # Remove the CWD from sys.path while we load stuff.


 "B01MDUOZAO_1, B01MDUOZAO_2, B01MDUOZAO_3, B01MDUOZAO_4, B01MDUOZAO_5, B01MDUOZAO_6, B01MDUOZAO_7, B01MDUOZAO_8, B01MDUOZAO_9, B01MDUOZAO_10", B00CL2H1U2_2, B00CL2H1U2_4, B00CL2H1U2_8, B00U8K2LHW_3, B00U8K2LHW_5, B00U8K2LHW_6, B00U8K2LHW_7, B00U8K2LHW_8, B00U8K2LHW_9, B00U8K2LHW_10", "B075S59W5K_1, B075S59W5K_2, B075S59W5K_3, B075S59W5K_4, B075S59W5K_5, B075S59W5K_6, B075S59W5K_7, B075S59W5K_8, B075S59W5K_9, B075S59W5K_10", "B00V7U69QQ_1, B00V7U69QQ_3, B00V7U69QQ_4, B00V7U69QQ_6, B00V7U69QQ_9, B00V7U69QQ_10", B00B9MU1DE_3, B00B9MU1DE_7, B0080AHHBM_7, B0080AHHBM_8, B01M7PAZ0M_2, B01M7PAZ0M_3, B01M7PAZ0M_6, B01M7PAZ0M_7, "B073WNQGC7_1, B073WNQGC7_4, B073WNQGC7_5, B073WNQGC7_6, B073WNQGC7_7, B073WNQGC7_8, B073WNQGC7_9, B073WNQGC7_10", B01JIJEPIQ_4, B01JIJEPIQ_9, "B071P954CM_1, B071P954CM_4, B071P954CM_5, B071P954CM_8, B071P954CM_9, B071P954CM_10", B01DU5M5T0_2, B01DU5M5T0_3, B01DU5M5T0_4, B01DU5M5T0_5, B01DU5M5T0_6, B01DU5M5T0_7, B01DU5M5T0_8, B01DU5M5T0_9, B01DU5M5T0_10", B072PTTM41_2, 

 B00O8RQYGU_5, B00O8RQYGU_8, B00O8RQYGU_9, B00KLXOZD4_4, B00KLXOZD4_5, B00KLXOZD4_7, B00KLXOZD4_8, B00KLXOZD4_10", "B00D9KSY8I_1, B00D9KSY8I_3, B00D9KSY8I_4, B00D9KSY8I_5, B00D9KSY8I_6, B00D9KSY8I_9, B00D9KSY8I_10", "B01A0FEPFU_1, B01A0FEPFU_2, B01A0FEPFU_3, B01A0FEPFU_4, B01A0FEPFU_5, B01A0FEPFU_6, B01A0FEPFU_7, B01A0FEPFU_8, B01A0FEPFU_9, B01A0FEPFU_10", B0756CG87G_2, B0756CG87G_8, B0756CG87G_9, B0756CG87G_10", "B074RFJQB2_1, B074RFJQB2_3, B074RFJQB2_4, B074RFJQB2_5, B074RFJQB2_6, B074RFJQB2_7, B074RFJQB2_8, B074RFJQB2_9, B074RFJQB2_10", B01B7GAGWM_2, B01B7GAGWM_4, B01B7GAGWM_5, B01B7GAGWM_6, B01B7GAGWM_7, B01B7GAGWM_8, B01B7GAGWM_9, B01B7GAGWM_10", "B00QT354D0_1, B00QT354D0_2, B00QT354D0_3, B00QT354D0_4, B00QT354D0_5, B00QT354D0_6, B00QT354D0_7, B00QT354D0_8, B00QT354D0_9, B00QT354D0_10", "B0194M6A3U_1, B0194M6A3U_3, B0194M6A3U_4, B0194M6A3U_5, B0194M6A3U_6, B0194M6A3U_9, B0194M6A3U_10", "B00OJG5MDG_1, B00OJG5MDG_6, B00OJG5MDG_7, B00OJG5MDG_8, B00OJG5MDG_9, B00OJG5MDG_10", B00QT354S

Cluster 5 titles: "B01MZALP0W_1, B01MZALP0W_8, B0194WFWCK_4, B0194WFWCK_9, B073FYF9S2_5, B00VR2E6AK_9, B073FWCJQV_5, "B015XA8YJK_1, "B01N34QXLH_1, B01N34QXLH_8, B018B2E65I_7, B01N2XLPXS_3, B00PESY1T4_4, B00OFR5H0M_2, B00OFR5H0M_3, B00OFR5H0M_9, "B00HH25GGW_1, "B00ME8CNYW_1, B00ME8CNYW_3, B00ME8CNYW_5, B00VR2E6O6_9, "B0713RPXF3_1, B0713RPXF3_8, "B01JZ6BWKQ_1, "B01N34QVF6_1, B01N34QVF6_8, "B01N6N3JYC_1, B01N6N3JYC_2, B01MY4HMFR_7, "B01N9SIV16_1, B01N9SIV16_8, B01I9GUD62_5, B00E94U12A_8, B00ZWTUHH0_3, B00ZWTUHH0_6, B00ZWTUHH0_10", B06X1J2LMH_3, B06X1J2LMH_4, B06X1J2LMH_10", B01G3A7A5S_5, B071XP2FKX_3, B00JM5GW10_2, B004SE3O70_2, B004SE3O70_7, B00TFWOS8Y_8, B000M25PE2_6, B000M25PE2_7, B000M25PE2_8, B01G1SB5I0_8, B01HPG6NVQ_8, B01HPG6NK2_8, B01HPG6NIE_8, B01MTO7JX7_5, B00COX3M4W_4, B00COX3M4W_10", B01I9GUBTQ_3, B01I9GUBTQ_6, B01I9GUBTQ_10", B010Q2CF0E_3, B010Q2CF0E_8, B016ZDDMFA_3, B016ZDDMFA_6, B016ZDDMFA_10", B0017IMBCO_4, B014H0UJ6I_5, B002YIRKKY_5, B002YIRKKY_7, B002YIRKKY_10", B00IGNX2

In [91]:
from __future__ import print_function
 
print("Top terms per cluster:")
print()
# 按离质心的距离排列聚类中心，由近到远
order_centroids = km.cluster_centers_.argsort()[:, ::-1] 
# print(order_centroids)
for i in range(num_clusters):
    print("Cluster %d:" % i, end='')
    time.sleep(1)
    for ind in order_centroids[i, :50]: # 每个聚类选 10 个词
        print(' %s' % word[ind]  , end=',')
    print() # 空行
    print() # 空行

Top terms per cluster:

Cluster 0: 做工, 玩具, 孩子, 喜欢, 亚马逊, 儿子, 感觉, 包装, 东西, 味道, 质量, 价格, 满意, 设计, 打开, 电池, 不错, 适合, 精致, 收到, 便宜, 好玩, 不好, 真的, 轨道, 商品, 失望, 简单, 一点, 只能, 正品, 火车, 粗糙, 太小, 娃娃, 牌子, 地方, 宝贝, 塑料, 盒子, 两个, 好看, 想象, 性价比, 发货, 玩儿, 图片, 小贵, 很大, 产品,

Cluster 1: 积分, 评价, 不用, 复制到, 复制, 换钱, 走到, 这段话, 省事, 既能, 浪费, 发出, 好好, 评论, 推荐, 有用吗, 留言, 飘飘, 锦衣, 侄女, 确认, 满意, 完美, 购买, 宝宝, 可爱, 做工, 孩子, 一岁, 颜色, 真的, 儿子, 礼物, 正品, 味道, 感觉, 想象, 开心, 不太好, 品牌, 很小, 爱不释手, 娃娃, 塑料, 东西, 火车, 外面, 姑娘, 小车, 用料,

Cluster 2: 可爱, 女儿, 喜欢, 颜色, 精致, 送人, 包装, 鲜艳, 不太好, 瑕疵, 很小, 异味, 好看, 火车, 太小, 感觉, 对得起, 做工, 孩子, 图片, 手感, 真的, 价格, 听到, 反馈, 味道, 很软, 小侄女, 舒服, 东西, 照片, 不错, 本来, 妈妈, 种类, 上深, 绒毛, 稀少, 蓝色, 入手, 质量, 小是, 速度, 发现, 玩具, 质感, 测试, 合适, 苗条, 坐姿,

Cluster 3: 宝宝, 喜欢, 不错, 做工, 可爱, 能力, 合适, 精细, 玩具, 质量, 毛绒, 锻炼, 拼图, 爱不释手, 适合, 我家, 价格, 宝贝, 爱玩, 简单, 柔软, 拿到, 两岁, 一岁, 一点, 小车, 配件, 大人, 亚马逊, 第一次, 介绍, 设计, 动手, 挖掘机, 期待, 睡觉, 娃娃, 时式, 包装纸, 放入, 撕扯, 盒内, 广告宣传, 分包, 大小, 小兔子, 咯手, 觉着, 作工, 晚上,

Cluster 4: 不错, 质量, 做工, 喜欢, 东西, 凑单, 价格, 孩子, 儿子, 满意, 制造, 好评, 太小, 性价比, 手感, 想象, 中国, 车子, 品牌, 爱玩, 亚马逊, 小车, 很棒, 颜色,

## LDA

In [92]:
from sklearn.decomposition import LatentDirichletAllocation

In [93]:
n_topics = 10
lda = LatentDirichletAllocation(n_topics=n_topics, max_iter=50,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)

In [113]:
%time L = lda.fit(tf) #tf 为向量化后的语料词集

/usr/local/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


CPU times: user 34.9 s, sys: 399 ms, total: 35.3 s
Wall time: 35.5 s


In [115]:
type(L)

sklearn.decomposition.online_lda.LatentDirichletAllocation

In [166]:
%time lda_matrix = lda.fit_transform(tf)

/usr/local/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


CPU times: user 36.1 s, sys: 526 ms, total: 36.6 s
Wall time: 37.2 s


In [167]:
lda_matrix

array([[ 0.02500027,  0.025     ,  0.025     , ...,  0.025     ,
         0.025     ,  0.77499973],
       [ 0.020011  ,  0.02      ,  0.02000212, ...,  0.0200003 ,
         0.02000436,  0.57462934],
       [ 0.1       ,  0.1       ,  0.1       , ...,  0.1       ,
         0.1       ,  0.1       ],
       ..., 
       [ 0.00769264,  0.00769231,  0.77692142, ...,  0.00769247,
         0.00769231,  0.16153815],
       [ 0.00588308,  0.00588235,  0.00588432, ...,  0.07251232,
         0.00588289,  0.00588235],
       [ 0.84998988,  0.01666667,  0.0166741 , ...,  0.01666694,
         0.01666791,  0.01666718]])

In [169]:
#https://www.pixpo.net/education/0IJdxJRZ.html
features = pd.DataFrame(lda_matrix, columns=['T1', 'T2', 'T3','T4','T5','T6','T7','T8','T9','T10'])

In [176]:
features[:5]

,T1,T2,T3,T4,T5,T6,T7,T8,T9,T10
0,0.025000,0.025,0.025000,0.025,0.025,0.025,0.025000,0.025,0.025000,0.775000
1,0.020011,0.020,0.020002,0.020,0.020,0.020,0.265353,0.020,0.020004,0.574629
2,0.100000,0.100,0.100000,0.100,0.100,0.100,0.100000,0.100,0.100000,0.100000
3,0.100000,0.100,0.100000,0.100,0.100,0.100,0.100000,0.100,0.100000,0.100000
4,0.100000,0.100,0.100000,0.100,0.100,0.100,0.100000,0.100,0.100000,0.100000


In [171]:
%time km.fit_transform(features)

CPU times: user 178 ms, sys: 13.9 ms, total: 192 ms
Wall time: 186 ms


array([[ 0.72015675,  0.99648933,  1.00146708, ...,  0.92343361,
         0.91835057,  0.9573937 ],
       [ 0.5604219 ,  0.89328248,  0.89550313, ...,  0.80595549,
         0.80235041,  0.84810172],
       [ 0.02984682,  0.61957949,  0.6311005 , ...,  0.50408285,
         0.48486424,  0.55262243],
       ..., 
       [ 0.73311318,  1.01944931,  1.02286552, ...,  0.93992854,
         0.9249662 ,  0.95885264],
       [ 0.72533017,  0.97232385,  1.01826739, ...,  0.23554938,
         0.93933672,  0.97528086],
       [ 0.7828694 ,  1.00514147,  0.16209411, ...,  0.9555793 ,
         0.90575711,  0.98729231]])

In [172]:
cluster_labels = km.labels_

In [173]:
cluster_labels = pd.DataFrame(cluster_labels, columns=['ClusterLabel'])

In [194]:
cluster_labels[:5]

,ClusterLabel
0,6
1,6
2,0
3,0
4,0


In [182]:
ldaframe = pd.DataFrame({ 'title': ASINs, 'rank': Ranks, 'review': Reviews,'category': Category })

In [183]:
ldaframe = ldaframe[['title','rank','category','review']]

In [206]:
ldaClusterframe = pd.concat([ldaframe, cluster_labels], axis=1)

In [207]:
ldaClusterframe[:5]

,title,rank,category,review,ClusterLabel
0,"""B06WWD7SMQ_1",5.0,1,头可动，模型也很逼真,6
1,B06WWD7SMQ_2,5.0,1,商品到达得很快，很不错~,6
2,B06WWD7SMQ_3,0.0,1,,0
3,B06WWD7SMQ_4,0.0,1,,0
4,B06WWD7SMQ_5,0.0,1,,0


In [208]:
ldaClusterframe['ClusterLabel'].value_counts()

0    2617
2    1061
3     539
1     441
4     352
5     344
6     308
7     235
8     206
9     187
Name: ClusterLabel, dtype: int64

In [209]:
Rankgrouped = ldaClusterframe['rank'].groupby(ldaClusterframe['ClusterLabel']) # 为了凝聚（aggregation），由聚类分类。
 
Rankgrouped.mean() # 每个聚类的平均排名（0 到 5）

ClusterLabel
0    0.639282
1    4.360544
2    4.634307
3    4.688312
4    4.585227
5    4.148256
6    4.074675
7    4.255319
8    4.233010
9    4.005348
Name: rank, dtype: float64

In [210]:
#排序
ldaClusterframe.sort_values(by = 'ClusterLabel')[:10]

,title,rank,category,review,ClusterLabel
3771,B00P8O2ED4_2,0.0,7,,0
2044,B01DU5M5T0_5,0.0,4,,0
2045,B01DU5M5T0_6,0.0,4,,0
2046,B01DU5M5T0_7,0.0,4,,0
2047,B01DU5M5T0_8,0.0,4,,0
2048,B01DU5M5T0_9,0.0,4,,0
2049,"B01DU5M5T0_10""",0.0,4,,0
2051,B072PTTM41_2,0.0,4,,0
2052,B072PTTM41_3,0.0,4,,0
2043,B01DU5M5T0_4,0.0,4,,0


In [214]:
#挑選特定條件
ldaClusterframe[ldaClusterframe.loc[:,'ClusterLabel'] == 6][:100]

,title,rank,category,review,ClusterLabel
0,"""B06WWD7SMQ_1",5.0,1,头可动，模型也很逼真,6
1,B06WWD7SMQ_2,5.0,1,商品到达得很快，很不错~,6
20,"""B00A30ZA34_1",4.0,1,比较小，容组装，感觉还好吗,6
53,B0063DEIQK_4,4.0,1,夹子内侧像橡皮的固定物时间长了就挤成渣了,6
64,B01MZALP0W_5,3.0,1,做工还行，但好几个都站不住，娃娃玩了一次就不想再玩了,6
70,"""B06XHJ1JRT_1",5.0,1,形象还原得不错，站得也挺稳。,6
150,"""B0194WFWCK_1",5.0,1,孩子送给同学的生日礼物，由于时间急，和卖家联系，卖家想用户所需，立即改发顺丰，第二天就收到了...,6
151,B0194WFWCK_2,4.0,1,人物做的比较像，很逼真，细节不错，关节要是能活动就更好了,6
154,B0194WFWCK_5,5.0,1,给小侄女买的圣诞节礼物，做工精致，脸部逼真，没有那种假娃娃连我的感觉,6
161,B073FYF9S2_2,5.0,1,感觉买贵了，出了贵，没毛病。好用，相信亚马逊。,6


## LDA 2

In [95]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic #%d:" % topic_idx)
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))
        print([model.components_[topic_idx][i-1] for i in topic.argsort()[:-n_top_words - 1:-1]])
        
    print()

In [96]:
n_top_words = 30

In [97]:
tf_feature_names = vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, n_top_words)

Topic #0:
不错 喜欢 质量 做工 玩具 孩子 宝宝 小孩 价格 儿子 味道 好玩 手感 设计 图片 东西 活动 异味 开心 精致 小车 爱玩 牌子 想象 划算 精细 超级 粗糙 购买 车子
[0.10000205279770037, 0.10001665478687394, 0.10000483516678697, 1.26161354509381, 0.10000853955073997, 0.10000383334418854, 0.10000000010106537, 0.10000745170863339, 0.10000000028605403, 0.10001045747005695, 0.10000000010505206, 0.10000987593750103, 0.10000293450685328, 0.10001429346663272, 0.10000456115625689, 0.10000000007839807, 0.100016759238349, 0.10000288368213384, 3.7350569930223951, 0.10002158202238789, 0.10005012287227573, 0.10000570052842774, 0.10000239145289629, 0.10000090502143714, 0.10000797039851365, 0.99375395312450099, 0.10000279814026966, 0.10000359716964982, 0.10003829508563876, 0.10000663974649709]
Topic #1:
难度 车厢 回家 系列 客服 照片 实物 小男孩 有时候 想要 卡扣 超过 情况 哇哇 几种 一摸 太软 半小时 用时 放在 太好了 绒毛 稀少 上深 紫霞 车子 组合 木屑 人员 吃惊
[0.10000000067838662, 0.10000000037997397, 0.10000000133263441, 0.10000000015437614, 0.10000000010562227, 0.10000000028869624, 1.1373740219254436, 0.10000000115507282, 0.1

In [98]:
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(lda, tf, vectorizer)

/usr/local/lib/python3.6/site-packages/pyLDAvis/_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]


PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
0      20.583109        1       1 -0.250624 -0.057981
7      15.310618        1       2 -0.073952  0.227755
8      12.289928        1       3 -0.088882 -0.139544
2      11.863946        1       4 -0.177105  0.084701
6      10.670002        1       5 -0.104063 -0.020879
9       7.399986        1       6  0.077133 -0.195164
3       6.444813        1       7  0.169906  0.144274
4       5.731755        1       8  0.128778 -0.004287
5       5.374650        1       9  0.157268 -0.025516
1       4.331193        1      10  0.161540 -0.013358, topic_info=     Category         Freq Term        Total  loglift  logprob
term                                                          
1903  Default   689.000000   孩子   689.000000  30.0000  30.0000
1433  Default  1133.000000   喜欢  1133.000000  29.0000  29.0000
306   Default   738.000000   不错   738.000000  28.0000  28.0000
4726  Default   521.000000   质量   521.000000  27.0000  27.0000
1929  Default   561.000000   宝宝   561.000000  26.0000  26.0000
707   Default   346.000000   做工   346.000000  25.0000  25.0000
3698  Default   287.000000   玩具   287.000000  24.0000  24.0000
2647  Default    97.000000   拼图    97.000000  23.0000  23.0000
2045  Default   150.000000  小朋友   150.000000  22.0000  22.0000
507   Default   242.000000  亚马逊   242.000000  21.0000  21.0000
3547  Default   114.000000   满意   114.000000  20.0000  20.0000
1067  Default   183.000000   包装   183.000000  19.0000  19.0000
2456  Default   146.000000   感觉   146.000000  18.0000  18.0000
5002  Default   136.000000   适合   136.000000  17.0000  17.0000
1283  Default   186.000000   可爱   186.000000  16.0000  16.0000
1729  Default    89.000000   太小    89.000000  15.0000  15.0000
4033  Default   120.000000   积分   120.000000  14.0000  14.0000
3668  Default   138.000000   特别   138.000000  13.0000  13.0000
3183  Default    74.000000   柔软    74.000000  12.0000  12.0000
730   Default   158.000000   儿子   158.000000  11.0000  11.0000
54    Default   129.000000   一岁   129.000000  10.0000  10.0000
4773  Default    84.000000   超级    84.000000   9.0000   9.0000
570   Default   168.000000   价格   168.000000   8.0000   8.0000
277   Default    97.000000   不用    97.000000   7.0000   7.0000
4650  Default    96.000000   评价    96.000000   6.0000   6.0000
1798  Default    77.000000   女儿    77.000000   5.0000   5.0000
652   Default    99.000000   便宜    99.000000   4.0000   4.0000
3938  Default   101.000000   真的   101.000000   3.0000   3.0000
3139  Default    62.000000   材质    62.000000   2.0000   2.0000
4141  Default    91.000000   简单    91.000000   1.0000   1.0000
...       ...          ...  ...          ...      ...      ...
2436  Topic10     9.446596   想要    10.210375   3.0616  -5.0654
1139  Topic10     9.104200   卡扣     9.868046   3.0588  -5.1024
4776  Topic10     8.736244   超过     9.500167   3.0555  -5.1436
2422  Topic10     8.566699   情况     9.330573   3.0539  -5.1632
1398  Topic10     8.327396   哇哇     9.091144   3.0516  -5.1915
899   Topic10     8.126949   几种     8.891068   3.0495  -5.2159
1756  Topic10     7.363009   太软     8.126819   3.0406  -5.3146
1104  Topic10     7.340944  半小时     8.104698   3.0404  -5.3176
3779  Topic10     6.574491   用时     7.338243   3.0294  -5.4279
1727  Topic10     6.383510  太好了     7.147266   3.0263  -5.4574
4254  Topic10     6.301502   绒毛     7.065254   3.0249  -5.4703
4041  Topic10     6.301502   稀少     7.065254   3.0249  -5.4703
188   Topic10     6.301502   上深     7.065254   3.0249  -5.4703
4204  Topic10     6.214983   紫霞     6.978732   3.0234  -5.4841
4234  Topic10     5.957895   组合     6.721712   3.0187  -5.5264
3110  Topic10     5.950459   木屑     6.714522   3.0185  -5.5276
531   Topic10     5.836674   人员     6.600425   3.0164  -5.5469
1295  Topic10     5.809654   吃惊     6.573524   3.0158  -5.5516
2768  Topic10     5.747387   提供     6.511146   3.0146  -5.5624
3793  Topic10     5.694216   电影  

/usr/local/Cellar/python3/3.6.1/Frameworks/Python.framework/Versions/3.6/lib/python3.6/json/encoder.py:199: DeprecationWarning: Interpreting naive datetime as local 2018-02-20 23:27:21.317417. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


In [232]:
data = pyLDAvis.sklearn.prepare(lda, tf, vectorizer)
pyLDAvis.show(data)

/usr/local/lib/python3.6/site-packages/pyLDAvis/_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]



Note: if you're in the IPython notebook, pyLDAvis.show() is not the best command
      to use. Consider using pyLDAvis.display(), or pyLDAvis.enable_notebook().
      See more information at http://pyLDAvis.github.io/quickstart.html .

You must interrupt the kernel to end this command

Serving to http://127.0.0.1:8889/    [Ctrl-C to exit]


127.0.0.1 - - [18/Feb/2018 19:53:54] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [18/Feb/2018 19:53:54] "GET /LDAvis.css HTTP/1.1" 200 -
127.0.0.1 - - [18/Feb/2018 19:53:54] "GET /d3.js HTTP/1.1" 200 -
127.0.0.1 - - [18/Feb/2018 19:53:54] "GET /LDAvis.js HTTP/1.1" 200 -



stopping Server...


/usr/local/Cellar/python3/3.6.1/Frameworks/Python.framework/Versions/3.6/lib/python3.6/json/encoder.py:199: DeprecationWarning: Interpreting naive datetime as local 2018-02-18 19:53:52.307771. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


In [233]:
pyLDAvis.disable_notebook()

/usr/local/Cellar/python3/3.6.1/Frameworks/Python.framework/Versions/3.6/lib/python3.6/json/encoder.py:199: DeprecationWarning: Interpreting naive datetime as local 2018-02-18 19:55:32.938488. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)
